<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_9_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [12]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    916      0 --:--:-- --:--:-- --:--:--   916
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [13]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

fatal: destination path 'demo-nlp-classification-fastai' already exists and is not an empty directory.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [19]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,3,cracking sensations in my joints
1,5,fast and slow heartbeat
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [20]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-9.test.csv",header=None)
test.head()

,0,1
0,2,painful heels
1,4,Discoloration of skin [blotchy] on the neck
2,5,my heart rate jumped to 106
3,7,goiter
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [25]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [29]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.458558,4.790451,0.232222,10:55


In [30]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.665819,4.483639,0.260793,10:50


In [31]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.365370,4.447103,0.264456,10:51


In [32]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.322684,4.421189,0.266886,10:52


In [0]:
learn.save_encoder('pubmed_fold9_first')

In [34]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_fold9_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.812557,3.379990,0.423181,01:10


In [35]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.912664,2.908961,0.487871,01:23


In [36]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.582572,2.674845,0.535040,03:33


In [37]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.302976,2.517119,0.563342,03:12
1,2.083496,2.436194,0.572776,03:10


In [38]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.989856,2.292997,0.578167,03:20
1,1.821084,2.237288,0.594340,03:15


In [39]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.804544,2.183010,0.610512,03:31
1,1.651192,2.117889,0.605121,03:20


In [40]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.638585,2.121274,0.607817,03:29
1,1.539298,2.077086,0.622642,03:18


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.559012,2.077672,0.613208,03:25
1,1.454754,2.029981,0.628032,03:28


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.408837,2.043693,0.637466,03:11
1,1.384676,2.015822,0.632075,03:18
2,1.312493,1.987773,0.634771,03:30
3,1.186308,1.974858,0.633423,03:16


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.281486,2.008261,0.650943,03:41
1,1.325211,1.997839,0.638814,03:18
2,1.199725,1.968548,0.660377,03:22
3,1.120100,1.959829,0.641509,03:13


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.158642,1.996521,0.649596,03:36
1,1.232939,1.982290,0.659030,03:39
2,1.167164,2.007324,0.649596,03:18
3,1.125383,1.998913,0.654987,03:14


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.153378,2.011880,0.648248,03:29
1,1.184224,2.110044,0.645553,03:33
2,1.163764,1.994829,0.650943,03:22
3,1.050304,1.978045,0.649596,03:28


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.136363,2.069120,0.644205,03:18
1,1.136181,2.115364,0.641509,03:18
2,1.102218,2.004879,0.650943,03:23
3,1.069470,2.024727,0.652291,03:28


In [47]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.080383,2.041875,0.650943,03:34
1,1.114323,2.055382,0.645553,03:15
2,1.072231,2.058078,0.652291,03:32
3,1.029426,2.007932,0.645553,03:20


In [48]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.071237,2.076070,0.645553,03:29
1,1.081683,2.059835,0.648248,03:17
2,1.054800,2.077930,0.656334,03:29
3,0.979047,2.040316,0.649596,03:35


In [0]:
# save the best model

learn.save_encoder('ask-9-tuning-pubmed')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [51]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[686, 348, 788, 37, 8, 8, 8, 8, 8, 164, 359, 359, 529, 318, 37, 879, 246, 27, 27, 750, 31, 31, 584, 635, 37, 37, 37, 194, 182, 794, 289, 289, 289, 37, 511, 462, 511, 49, 49, 49, 775, 245, 53, 53, 343, 830, 194, 63, 426, 245, 71, 127, 76, 76, 893, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 265, 686, 37, 37, 37, 37, 636, 182, 289, 80, 80, 491, 448, 189, 108, 88, 627, 89, 37, 37, 37, 92, 94, 94, 94, 94, 94, 94, 94, 94, 228, 681, 37, 716, 746, 194, 49, 100, 100, 100, 575, 712, 108, 108, 108, 384, 111, 111, 111, 111, 117, 194, 117, 194, 125, 630, 630, 630, 630, 125, 125, 125, 125, 125, 125, 125, 125, 125, 343, 129, 127, 511, 27, 268, 268, 137, 37, 499, 142, 449, 37, 37, 158, 1032, 1032, 321, 37, 37, 37, 37, 37, 37, 245, 172, 289, 723, 704, 289, 172, 172, 172, 470, 37, 624, 165, 904, 847, 194, 182, 289, 37, 646, 646, 646, 183, 183, 183, 37, 37, 37, 37, 188, 190, 190, 190, 194, 211, 289, 289, 197, 197, 202, 260, 210,

In [52]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

452
0.5219399538106235
